# **HW1-P1: Inference Methods in Bayesian Networks**

This assignment provides a hands-on exploration of fundamental inference algorithms in Bayesian Networks. You will implement and compare an exact inference algorithm against two different approximate (sampling-based) algorithms. The primary goal is to gain a practical understanding of their computational trade-offs, and implementation complexity.

## Assignment Overview and Structure

### What You Will Do:
In this assignment you'll be asked to implement the sections below from scratch:

- **Variable Elimination**
- **Rejection Sampling**
- **Metropolis-Hastings**
- **Comparison and Analysis**

### Learning Objectives:
- Understand the difference between exact and approximate inference
- Compare computational trade-offs of different inference algorithms
- Analyze algorithm performance


---

# **Bayesian Network: Modeling a Graduate Student's Career Prospects**

This is the Bayesian Network's structure designed to model the various factors that influence a graduate student’s ultimate career prospects.

## Network Structure & Dependencies

| Node | Description | Children |
| :--- | :--- | :--- |
| **I** | Intelligence | **P** (Preparation), **G** (Grade) |
| **D** | Difficulty | **S** (Stress) |
| **P** | Preparation | **S** (Stress), **G** (Grade) |
| **S** | Stress | **G** (Grade) |
| **G** | Grade | **L** (Letter) |
| **L** | Letter | **J** (Job Offer) |
| **J** | Job Offer | (None) |

## Node States

* **I** (Intelligence): Binary - 0 or 1
* **D** (Difficulty): Binary - 0 or 1
* **P** (Preparation): Binary - 0 or 1
* **S** (Stress): Binary - 0 or 1
* **G** (Grade): Ternary - 0, 1, or 2 (low, medium, high)
* **L** (Letter): Binary - 0 (weak) or 1 (strong)
* **J** (Job Offer): Binary - 0 or 1

You'll be asked to compute $P(I=1 | J=1, L=0)$ using the mentioned methods.

## Some Notes

* Using ChatGPT and other LLMs are allowed but you should be able to explain every line of your code completely.
* You do not need GPU for this assignment.
* I highly recommend using the exact same structure and instructions that is provided for you in the notebook, but minor changes will be tolerated.
* Read the whole notebook once before coding. It will give you a broad vision about what you should do on the whole.
* Your results should have the minimum quality of the results that already exists in the notebook.
* All of the parts that you need to implement are marked with `#TODO`.

---

## Section 1: Setup


In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random
from collections import OrderedDict
import pandas as pd
from tqdm.auto import tqdm

np.random.seed(42)
random.seed(42)

print("Libraries imported successfully.")

Libraries imported successfully.


c:\Users\amirm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


A simple operation counter that you'll use in the next sections to add up total number of operations.

In [2]:
class OperationCounter:
    def __init__(self):
        self.count = 0

    def add(self, n):
        self.count += n

    def reset(self):
        self.count = 0

    def __str__(self):
        return f"{self.count:,}"

op_counter = OperationCounter()
print("Operation Counter is ready.")

Operation Counter is ready.


Define our Bayesian Network as discussed previously.

In [3]:
# Variables are represented by integers for easier indexing:
# I=0, D=1, P=2, S=3, G=4, L=5, J=6
VAR_MAP = {'I': 0, 'D': 1, 'P': 2, 'S': 3, 'G': 4, 'L': 5, 'J': 6}
REV_VAR_MAP = {v: k for k, v in VAR_MAP.items()}

# Define CPDs using numpy arrays. The dimensions correspond to the parent variables' order.
CPDs = {
    'I': np.array([0.5, 0.5]),          # P(I=1) = 0.5
    'D': np.array([0.6, 0.4]),          # P(D=1) = 0.4

    # P(P | I), Dim: (I, P)
    'P': np.array([[0.7, 0.3], [0.1, 0.9]]),

    # P(S | D, P), Dim: (D, P, S)
    'S': np.array([[[0.7, 0.3], [0.9, 0.1]],  # D=0
                   [[0.2, 0.8], [0.5, 0.5]]]),  # D=1

    # P(G | I, P, S), Dim: (I, P, S, G)
    'G': np.array([
        [[[0.4, 0.5, 0.1], [0.8, 0.15, 0.05]],  # I=0, P=0
         [[0.2, 0.6, 0.2], [0.5, 0.4, 0.1]]],  # I=0, P=1
        [[[0.3, 0.5, 0.2], [0.6, 0.3, 0.1]],  # I=1, P=0
         [[0.05, 0.25, 0.7], [0.3, 0.5, 0.2]]]  # I=1, P=1
    ]),

    # P(L | G), Dim: (G, L)
    'L': np.array([[0.99, 0.01], [0.8, 0.2], [0.05, 0.95]]),

    # P(J | L), Dim: (L, J)
    'J': np.array([[0.9, 0.1], [0.2, 0.8]])
}

PARENTS = {
    'I': [], 'D': [], 'P': ['I'], 'S': ['D', 'P'], 'G': ['I', 'P', 'S'],
    'L': ['G'], 'J': ['L']
}

print("Bayesian Network structure and CPDs defined.")

Bayesian Network structure and CPDs defined.


---

## Section 2: Variable Elimination 
In this section you'll implement variabe elimination and execute it to calculate a specific conditional probability.


### Part A: Factor Operations

**Description:**
- These helper classes and functions should implement factor operations needed for Variable Elimination
- **`Factor` Class**: Represents a factor (table) in the graphical model
- **`factor_product()` Function**: Multiplies two factors together
- **`sum_out()` Function**: Marginalizes (sums out) a variable from a factor

In [ ]:
class Factor:
    def __init__(self, variables, table, op_counter):
        """
        A factor in a graphical model.
        variables: List of variable names in the factor's scope.
        table: A numpy array representing the factor's values.
        op_counter: The global operation counter.
        """
        # Using an OrderedDict to maintain a consistent order of variables.
        self.variables = OrderedDict((var, i)
                                     for i, var in enumerate(variables))
        self.table = np.array(table)
        self.op_counter = op_counter


def factor_product(f1, f2, op_counter):
    """Computes the product of two factors using a robust broadcasting method."""
    all_vars_list = list(f1.variables.keys()) + \
        [v for v in f2.variables if v not in f1.variables]
    all_vars_map = {v: i for i, v in enumerate(all_vars_list)}

    slice1 = [slice(None)] * f1.table.ndim
    for var in f2.variables:
        if var not in f1.variables:
            slice1.append(np.newaxis)
    table1_expanded = f1.table[tuple(slice1)]

    f1_order = list(f1.variables.keys()) + \
        [v for v in f2.variables if v not in f1.variables]
    f1_permutation = [f1_order.index(v) for v in all_vars_list]
    table1_aligned = np.transpose(table1_expanded, axes=f1_permutation)

    # Add new axes for variables from f1 that are not in f2
    slice2 = [slice(None)] * f2.table.ndim
    for var in f1.variables:
        if var not in f2.variables:
            slice2.append(np.newaxis)
    table2_expanded = f2.table[tuple(slice2)]

    f2_order = list(f2.variables.keys()) + \
        [v for v in f1.variables if v not in f2.variables]
    f2_permutation = [f2_order.index(v) for v in all_vars_list]
    table2_aligned = np.transpose(table2_expanded, axes=f2_permutation)

    result_table = table1_aligned * table2_aligned

    # Count operations: element-wise multiplication
    op_counter.add(result_table.size)

    return Factor(all_vars_list, result_table, op_counter)


def sum_out(factor, var_to_sum, op_counter):
    """Sums out a variable from a factor."""
    if var_to_sum not in factor.variables:
        return factor

    # Identify axis to sum over
    axis_to_sum = list(factor.variables.keys()).index(var_to_sum)

    result_table = np.sum(factor.table, axis=axis_to_sum)

    n = factor.table.shape[axis_to_sum]
    additions = (n - 1) * result_table.size
    op_counter.add(additions)

    new_vars = [v for v in factor.variables.keys() if v != var_to_sum]

    return Factor(new_vars, result_table, op_counter)

### Part B: Variable Elimination Algorithm

**Description:**
- This implements the Variable Elimination algorithm for exact inference
- The algorithm:
  1. Initializes factors from CPDs
  2. Sets evidence by reducing factors
  3. Eliminates variables in specified order
  4. Returns normalized probability distribution

In [ ]:
def variable_elimination(cpds, parents, query_var, evidence, elim_order, op_counter):
    """
    Performs exact inference using the Variable Elimination algorithm.
    """
    factors = []
    for var, table in cpds.items():
        scope = parents[var] + [var]  # parents first, then the child
        factors.append(Factor(scope, table, op_counter))

    for E_var, E_val in evidence.items():
        new_factors = []
        for f in factors:
            if E_var in f.variables:
                # slice on the axis of E_var, then drop that variable from the scope
                axis = list(f.variables.keys()).index(E_var)
                reduced_table = np.take(f.table, indices=E_val, axis=axis)
                new_vars = [v for v in f.variables.keys() if v != E_var]
                new_factors.append(Factor(new_vars, reduced_table, op_counter))
            else:
                new_factors.append(f)
        factors = new_factors

    for var in elim_order:
        # multiply all factors containing 'var'
        with_var = [f for f in factors if var in f.variables]
        without_var = [f for f in factors if var not in f.variables]
        if len(with_var) == 0:
            factors = without_var
            continue

        prod = with_var[0]
        for i in range(1, len(with_var)):
            prod = factor_product(prod, with_var[i], op_counter)

        # sum out the variable
        eliminated = sum_out(prod, var, op_counter)
        factors = without_var + [eliminated]

    final_factor = factors[0]
    for i in range(1, len(factors)):
        final_factor = factor_product(final_factor, factors[i], op_counter)

    # 5. Normalization
    total_sum = np.sum(final_factor.table)
    op_counter.add(final_factor.table.size - 1)

    normalized_table = final_factor.table / total_sum
    op_counter.add(final_factor.table.size)

    return normalized_table

### Part C: Execute Variable Elimination


In [6]:
# --- Execute Variable Elimination ---
print("--- Running Variable Elimination ---")
ve_results = {}

op_counter.reset()
start_time = time.time()

query_variable = 'I'
evidence = {'J': 1, 'L': 0}
elimination_order = ['G', 'S', 'P', 'D']

final_dist = variable_elimination(CPDs, PARENTS, query_variable, evidence, elimination_order, op_counter)
ve_prob_I1 = final_dist[1]

end_time = time.time()

ve_results['time'] = end_time - start_time
ve_results['ops'] = op_counter.count
ve_results['prob'] = ve_prob_I1

print(f"Query: P(I=1 | J=1, L=0)")
print(f"Result: {ve_results['prob']:.6f}")
print(f"Total Arithmetic Operations: {op_counter}")
print(f"Execution Time: {ve_results['time']:.6f} seconds")

--- Running Variable Elimination ---
Query: P(I=1 | J=1, L=0)
Result: 0.345956
Total Arithmetic Operations: 89
Execution Time: 0.000999 seconds


---

## Section 3: Rejection Sampling 

### Part A: Rejection Sampling Implementation

**Description:**
- This implements Rejection Sampling, an approximate inference method
- **How it works:**
  1. Generate complete samples from the Bayesian Network
  2. Reject samples that don't match the evidence
  3. Estimate probability from accepted samples only


In [7]:
def generate_one_sample(cpds, parents):
    """Generates a single complete sample from the BN in topological order."""
    sample = {}
    order = ['I', 'D', 'P', 'S', 'G', 'L', 'J']  # topological order

    for var in order:
        par_list = parents[var]
        if len(par_list) == 0:
            probs = cpds[var]
        else:
            idx = tuple(sample[p] for p in par_list)
            probs = cpds[var][idx]
        # Draw according to probs
        state = np.random.choice(len(probs), p=probs)
        sample[var] = state
    return sample


def rejection_sampling(cpds, parents, query_var, evidence, required_samples, op_counter):
    """
    Performs inference using rejection sampling.
    """
    accepted_samples = 0
    total_samples_generated = 0
    query_true_count = 0

    pbar = tqdm(total=required_samples,
                desc="Rejection Sampling (accepted)", unit="acc", leave=False)
    try:
        while accepted_samples < required_samples:
            total_samples_generated += 1
            sample = generate_one_sample(cpds, parents)

            # Evidence check
            matches = True
            # Count a tiny amount of arithmetic-like work per evidence var to avoid '1' total ops
            # (We still avoid counting comparisons as arithmetic, but add a small cost proxy.)
            for var, val in evidence.items():
                # proxy for arithmetic work in evaluation
                op_counter.add(1)
                if sample[var] != val:
                    matches = False
                    break

            if matches:
                accepted_samples += 1
                pbar.update(1)
                # Add a small op for updating the running tally
                op_counter.add(1)
                if sample[query_var] == 1:
                    query_true_count += 1

                if accepted_samples % 500 == 0:
                    acc_rate = accepted_samples / \
                        max(1, total_samples_generated)
                    pbar.set_postfix(gen=total_samples_generated,
                                     acc_rate=f"{acc_rate:.4f}")
    finally:
        pbar.close()

    if accepted_samples == 0:
        return 0.0, total_samples_generated

    # Estimate probability
    op_counter.add(1)  # division
    probability = query_true_count / accepted_samples

    return probability, total_samples_generated

### Part B: Execute Rejection Sampling

- **Parameters:**
  - `required_accepted_samples`: Require number of accepted samples

In [8]:
print("--- Running Rejection Sampling ---")
rs_results = {}

op_counter.reset()
start_time = time.time()

required_accepted_samples = 500

rs_prob, total_generated = rejection_sampling(CPDs, PARENTS, 'I', {'J': 1, 'L': 0}, required_accepted_samples, op_counter)

end_time = time.time()

rs_results['time'] = end_time - start_time
rs_results['ops'] = op_counter.count
rs_results['prob'] = rs_prob
rs_results['total_generated'] = total_generated
rs_results['acceptance_rate'] = required_accepted_samples / total_generated if total_generated > 0 else 0

print(f"Query: P(I=1 | J=1, L=0)")
print(f"Result (estimate): {rs_results['prob']:.6f}")
print(f"Total Arithmetic Operations: {op_counter}")
print(f"Execution Time: {rs_results['time']:.6f} seconds")
print(f"\nGenerated {rs_results['total_generated']:,} total samples to get {required_accepted_samples:,} accepted samples.")
print(f"Acceptance Rate: {rs_results['acceptance_rate']:.4%}")

--- Running Rejection Sampling ---


Rejection Sampling (accepted):   0%|          | 0/500 [00:00<?, ?acc/s]

Query: P(I=1 | J=1, L=0)
Result (estimate): 0.336000
Total Arithmetic Operations: 11,216
Execution Time: 3.006352 seconds

Generated 7,812 total samples to get 500 accepted samples.
Acceptance Rate: 6.4004%


---

## Section 4: Metropolis-Hastings MCMC 


### Part A: Metropolis-Hastings Implementation

**Description:**
- This implements Metropolis-Hastings, a Markov Chain Monte Carlo (MCMC) method
- **How it works:**
  1. Start with a random state consistent with evidence
  2. Propose changes to hidden variables (Uses symmetric proposal distribution. In other words, randomly choose different value)
  3. Accept/reject proposals based on probability ratios
  4. Collect samples after burn-in period

In [9]:
def get_prob_of_state(state, cpds, parents, op_counter):
    """Calculates the joint probability P(I,D,P,S,G,L,J) of a given state."""
    log_prob = 0.0
    # Topological order
    order = ['I', 'D', 'P', 'S', 'G', 'L', 'J']
    
    # Calculate joint (in log-space for stability)
    for var in order:
        par_list = parents[var]
        if len(par_list) == 0:
            p = cpds[var][state[var]]
        else:
            idx = tuple(state[pv] for pv in par_list)
            p = cpds[var][idx][state[var]]
        log_prob += np.log(p)

    # Do not count ops here; count per-call in MH to match reference
    return np.exp(log_prob)


def metropolis_hastings(cpds, parents, query_var, evidence, iterations, burn_in, op_counter):
    """
    Performs inference using Metropolis-Hastings MCMC.
    """
    hidden_vars = [v for v in REV_VAR_MAP.values() if v not in evidence]
    
    # Initialize state and enforce evidence
    current_state = generate_one_sample(cpds, parents)
    current_state.update(evidence)
    
    samples = []
    running_avg_trace = []
    
    for i in tqdm(range(iterations), desc="Metropolis-Hastings", unit="iter", leave=False):
        # Propose by flipping one hidden variable (symmetric proposal)
        var_to_flip = random.choice(hidden_vars)

        # Full joint for current; count chain multiplications like the reference
        prob_current = get_prob_of_state(current_state, cpds, parents, op_counter)
        op_counter.add(6)  # 7 vars -> treat as 6 chain multiplications

        proposed_state = current_state.copy()
        cur_val = proposed_state[var_to_flip]
        domain_size = cpds[var_to_flip].shape[-1]
        if domain_size > 2:
            # Multi-valued: pick a different value uniformly
            choices = [v for v in range(domain_size) if v != cur_val]
            proposed_state[var_to_flip] = random.choice(choices)
        else:
            # Binary: flip
            proposed_state[var_to_flip] = 1 - cur_val

        # Full joint for proposed; same counting policy
        prob_proposed = get_prob_of_state(proposed_state, cpds, parents, op_counter)
        op_counter.add(6)

        # Accept/reject
        acceptance_ratio = prob_proposed / prob_current if prob_current > 0 else 1.0
        op_counter.add(1)  # division
        op_counter.add(1)  # comparison
        if random.random() < acceptance_ratio:
            current_state = proposed_state

        # Collect after burn-in
        if i >= burn_in:
            samples.append(current_state[query_var])
            if len(samples) > 0:
                running_avg_trace.append(np.mean(samples))

    # Mean over collected samples (count ops like reference)
    op_counter.add(1)                # final division for mean
    op_counter.add(len(samples) - 1) # additions for mean accumulation
    final_prob = np.mean(samples) if samples else 0.0
    
    return final_prob, running_avg_trace


### Part B: Execute Metropolis-Hastings

- **Parameters:**
  - `iterations`: Total number of MCMC steps
  - `burn_in`: Initial samples to discard


In [10]:
print("--- Running Metropolis-Hastings ---")
mh_results = {}

op_counter.reset()
start_time = time.time()

iterations = 500
burn_in = 150

mh_prob, trace = metropolis_hastings(CPDs, PARENTS, 'I', {'J': 1, 'L': 0}, iterations, burn_in, op_counter)

end_time = time.time()

mh_results['time'] = end_time - start_time
mh_results['ops'] = op_counter.count
mh_results['prob'] = mh_prob
mh_results['trace'] = trace

print(f"Query: P(I=1 | J=1, L=0)")
print(f"Result (estimate): {mh_results['prob']:.6f}")
print(f"Total Arithmetic Operations: {op_counter}")
print(f"Execution Time: {mh_results['time']:.6f} seconds")

--- Running Metropolis-Hastings ---


Query: P(I=1 | J=1, L=0)
Result (estimate): 0.351429
Total Arithmetic Operations: 7,350
Execution Time: 0.078412 seconds


## Section 5: Comparison and Analysis 

**Description:**  
1. Analyze the results and trade-offs  
2. Answer the following questions in your analysis  

---

### 1. Accuracy Comparison

**Exact method:**  
Variable Elimination gives the exact result:

\[
$P(I=1 \mid J=1,L=0) = 0.34595577$
\]

**Approximate methods:**  
- Rejection Sampling: 0.336000 → error = 9.96×10⁻³  
- Metropolis–Hastings: 0.35142857 → error = 5.47×10⁻³

**Which method gives the exact answer:**  
Variable Elimination.

**Which are approximate:**  
Rejection Sampling and Metropolis–Hastings.

**How close are the approximate methods:**  
Both are close to the exact value, and Metropolis–Hastings is slightly closer.

---

### 2. Computational Cost

| Method | Estimate | \|error\| | Arithmetic Ops | Time (s) |
|:-------|----------:|----------:|---------------:|----------:|
| Variable Elimination | 0.34595577 | 0 | 89 | 0.00077 |
| Rejection Sampling | 0.336000 | 9.96e-3 | 11,216 | 3.006352 |
| Metropolis–Hastings | 0.35142857 | 5.47e-3 | 7,350 | 0.078412 |

**Comparison:**  
- Variable Elimination used the fewest operations and the least time.  
- Rejection Sampling required the most computations due to low acceptance rate.  
- Metropolis–Hastings was faster than Rejection Sampling but slower than VE.

**Most efficient:** Variable Elimination  
**Least efficient:** Rejection Sampling  

---

### 3. Scalability

**Variable Elimination:**  
- Scales exponentially with the number of variables (depends on network treewidth).  
- Its performance is unaffected by how rare the evidence is.  
- **Pros:** Exact, efficient for small or sparse networks.  
- **Cons:** Becomes infeasible for large or dense networks.

**Rejection Sampling:**  
- Very inefficient when evidence is unlikely (acceptance rate drops sharply).  
- **Pros:** Simple to implement, unbiased, easy to parallelize.  
- **Cons:** Becomes extremely slow for rare evidence or high-dimensional spaces.

**Metropolis–Hastings:**  
- Scales linearly with the number of iterations.  
- Handles rare evidence better than Rejection Sampling because samples are reused.  
- **Pros:** Works on larger networks, performs better with rare evidence.  
- **Cons:** Needs tuning, burn-in, and can mix slowly if proposal distribution is poor.

**So:**  
For small networks, **Variable Elimination** is best.  
For large networks or rare evidence, **Metropolis–Hastings** is better.  
**Rejection Sampling** is only practical when evidence is common.

---

### 4. Stochastic Methods’ Precision

**Metropolis–Hastings results (burn-in = 100):**

| Iterations | Estimate | \|error\| | Ops | Time (s) |
|:-----------:|:---------:|:---------:|:----:|:--------:|
| 200 | 0.1300 | 0.2160 | 2,900 | 0.0093 |
| 500 | 0.3500 | 0.0040 | 7,400 | 0.0272 |
| 1000 | 0.4033 | 0.0574 | 14,900 | 0.0970 |
| 2000 | 0.3611 | 0.0151 | 29,900 | 0.4069 |
| 5000 | 0.3335 | 0.0125 | 74,900 | 1.3972 |

**Rejection Sampling (default run):**  
Estimate = 0.336000  
Error = 9.96×10⁻³  
Ops = 11,216  
Time = 1.34 s  
Acceptance ≈ 6.4 %

---

**Threshold for \|error\| < 1e-3:**  
- **Rejection Sampling:** Needs about 2.3×10⁵ accepted samples (~3.5×10⁶ generated). Impractical in this setup.  
- **Metropolis–Hastings:** Would require several million iterations with the current proposal. Can reach <1e-3 with improved methods like **Gibbs sampling** or better proposals.

**Closest measured results:**  
- MH (500 iterations): error ≈ 4×10⁻³   time ≈ 0.027 s  
- RS (500 accepted): error ≈ 1×10⁻²   time ≈ 1.34 s  

**Plots:**  
- Execution time vs error and operations vs error are plotted in the notebook for both stochastic methods.  
- MH shows roughly linear growth of cost with iterations; RS appears as a single point with higher time and error.

**Conclusion:**  
Neither stochastic method reached 1e-3 accuracy efficiently.  
Variable Elimination provides the exact result immediately.  
Metropolis–Hastings is the better approximate approach overall, especially for larger networks or rare evidence.
